<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250730/%E3%82%AA%E3%83%AA%E3%82%A8%E3%83%B3%E3%83%86%E3%83%BC%E3%83%AA%E3%83%B3%E3%82%B0%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#準備


In [ ]:
%%capture
# ライブラリのインストール
!pip install mip
!pip install japanize-matplotlib

-----

## 演習問題1

例題で、スコア地点1 (Spot A) のスコアを50に増加させた場合、獲得スコアの最大値を求めよ。

###数理モデルの定式化

この問題は目的関数を以下のように変更する。

$$\text{Maximize} \quad Z = 50 y_1 + 20 y_2$$

その他の制約条件は例題と同一である。

###Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, maximize, BINARY, CONTINUOUS, OptimizationStatus
import math
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np

# --- データ定義 ---
# 地点データ: Spot Aのスコアを50に変更
nodes_data = {
    0: {'name': 'Start', 'coords': (0, 0), 'prize': 0, 'type': 'StartEnd'},
    1: {'name': 'Spot A', 'coords': (1, 3), 'prize': 50, 'type': 'Prize'}, # スコアを50に変更
    2: {'name': 'Spot B', 'coords': (4, 4), 'prize': 20, 'type': 'Prize'},
    3: {'name': 'End', 'coords': (5, 1), 'prize': 0, 'type': 'StartEnd'}
}

node_indices = list(nodes_data.keys())
num_nodes = len(node_indices)
start_node = 0
end_node = 3

prize_nodes = [i for i, data in nodes_data.items() if data['type'] == 'Prize']
prizes = {i: nodes_data[i]['prize'] for i in prize_nodes}

# 移動時間行列の計算 (ユークリッド距離)
travel_times = {}
for i in node_indices:
    for j in node_indices:
        if i == j: continue
        coord1 = nodes_data[i]['coords']
        coord2 = nodes_data[j]['coords']
        travel_times[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

T_max = 9.0  # 総移動時間上限
M = T_max + max(travel_times.values()) + 1 # MTZ制約用の大きな数

# --- モデル作成 ---
model = Model("Orienteering Problem - Ex1")

# 決定変数
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}")
     for i in node_indices for j in node_indices if i != j}
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in prize_nodes}
u = {i: model.add_var(lb=0, ub=T_max, name=f"u_{i}") for i in node_indices}

# 目的関数: 獲得スコアの最大化
model.objective = maximize(xsum(prizes[i] * y[i] for i in prize_nodes))

# --- 制約条件 ---
# 1. スタート/エンド制約
model += xsum(x[start_node, j] for j in node_indices if j != start_node) == 1
model += xsum(x[i, end_node] for i in node_indices if i != end_node) == 1
model += xsum(x[j, start_node] for j in node_indices if j != start_node) == 0
model += xsum(x[end_node, i] for i in node_indices if i != end_node) == 0

# 2. フロー保存と訪問決定
for k in prize_nodes:
    model += xsum(x[i, k] for i in node_indices if i != k) == y[k]
    model += xsum(x[k, j] for j in node_indices if j != k) == y[k]

# 3. 時間累積と部分巡回路除去 (MTZ)
model += u[start_node] == 0
for i in node_indices:
    for j in node_indices:
        if i != j and j != start_node:
            model += u[j] >= u[i] + travel_times[i, j] - M * (1 - x[i, j])

# --- 最適化の実行 ---
status = model.optimize(max_seconds=60)

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    total_prize = model.objective_value
    print(f"獲得総スコア: {total_prize:.2f}")

    # 経路の復元
    optimal_path = [start_node]
    current_node = start_node
    while current_node != end_node:
        for j in node_indices:
            if j != current_node and (current_node, j) in x and x[current_node, j].x >= 0.99:
                optimal_path.append(j)
                current_node = j
                break

    total_time = u[end_node].x
    print(f"最適経路: {optimal_path}")
    print(f"総移動時間: {total_time:.3f} (上限: {T_max})")

    # 到着時刻の表示
    print("各地点の訪問時刻:")
    for i in optimal_path:
        print(f"  ノード {i} ({nodes_data[i]['name']}): 到着時刻 {u[i].x:.3f}")

    # --- 結果の図示 ---
    plt.figure(figsize=(10, 7))
    # 地点プロット
    for i, data in nodes_data.items():
        color = 'lightgray'
        if i in optimal_path:
            if data['type'] == 'Prize':
                color = 'red'
            else: # Start/End
                color = 'gold'
        plt.scatter(data['coords'][0], data['coords'][1], s=200, color=color, edgecolors='black', zorder=5)
        plt.text(data['coords'][0] + 0.1, data['coords'][1] + 0.1, f"{i}: {data['name']}\n(p={data['prize']})")

    # 経路プロット
    path_coords = np.array([nodes_data[node_idx]['coords'] for node_idx in optimal_path])
    plt.plot(path_coords[:, 0], path_coords[:, 1], 'b-o', alpha=0.6, label="最適経路")

    plt.title(f"演習問題1の解\n総スコア: {total_prize:.0f}, 総時間: {total_time:.2f}")
    plt.xlabel("X座標")
    plt.ylabel("Y座標")
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()

elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

-----

## 演習問題2

例題の設定に戻し、総移動時間の上限を10にした場合、獲得スコアの最大値を求めよ。

###数理モデルの定式化

例題のスコア設定に戻し、総移動時間の上限 $T_{max}$ を 10 に変更します。


###Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, maximize, BINARY, CONTINUOUS, OptimizationStatus
import math
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np

# --- データ定義 ---
# 例題のデータ設定
nodes_data = {
    0: {'name': 'Start', 'coords': (0, 0), 'prize': 0, 'type': 'StartEnd'},
    1: {'name': 'Spot A', 'coords': (1, 3), 'prize': 10, 'type': 'Prize'},
    2: {'name': 'Spot B', 'coords': (4, 4), 'prize': 20, 'type': 'Prize'},
    3: {'name': 'End', 'coords': (5, 1), 'prize': 0, 'type': 'StartEnd'}
}

node_indices = list(nodes_data.keys())
num_nodes = len(node_indices)
start_node = 0
end_node = 3

prize_nodes = [i for i, data in nodes_data.items() if data['type'] == 'Prize']
prizes = {i: nodes_data[i]['prize'] for i in prize_nodes}

# 移動時間行列の計算
travel_times = {}
for i in node_indices:
    for j in node_indices:
        if i == j: continue
        coord1 = nodes_data[i]['coords']
        coord2 = nodes_data[j]['coords']
        travel_times[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

T_max = 10.0  # 総移動時間上限を10に変更
M = T_max + max(travel_times.values()) + 1

# --- モデル作成 ---
model = Model("Orienteering Problem - Ex2")

# 決定変数
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}")
     for i in node_indices for j in node_indices if i != j}
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in prize_nodes}
u = {i: model.add_var(lb=0, ub=T_max, name=f"u_{i}") for i in node_indices}

# 目的関数
model.objective = maximize(xsum(prizes[i] * y[i] for i in prize_nodes))

# --- 制約条件 ---
# (制約は演習問題1と同一のため省略)
model += xsum(x[start_node, j] for j in node_indices if j != start_node) == 1
model += xsum(x[i, end_node] for i in node_indices if i != end_node) == 1
model += xsum(x[j, start_node] for j in node_indices if j != start_node) == 0
model += xsum(x[end_node, i] for i in node_indices if i != end_node) == 0
for k in prize_nodes:
    model += xsum(x[i, k] for i in node_indices if i != k) == y[k]
    model += xsum(x[k, j] for j in node_indices if j != k) == y[k]
model += u[start_node] == 0
for i in node_indices:
    for j in node_indices:
        if i != j and j != start_node:
            model += u[j] >= u[i] + travel_times[i, j] - M * (1 - x[i, j])

# --- 最適化の実行 ---
status = model.optimize(max_seconds=60)

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    total_prize = model.objective_value
    print(f"獲得総スコア: {total_prize:.2f}")

    # 経路の復元
    optimal_path = [start_node]
    current_node = start_node
    while current_node != end_node:
        found_next = False
        for j in node_indices:
            if j != current_node and (current_node, j) in x and x[current_node, j].x >= 0.99:
                optimal_path.append(j)
                current_node = j
                found_next = True
                break
        if not found_next: break

    total_time = u[end_node].x
    print(f"最適経路: {optimal_path}")
    print(f"総移動時間: {total_time:.3f} (上限: {T_max})")
    print("訪問したスコア地点:", [i for i in prize_nodes if y[i].x >= 0.99])

    # --- 結果の図示 ---
    plt.figure(figsize=(10, 7))
    for i, data in nodes_data.items():
        color = 'lightgray'
        if i in optimal_path:
            color = 'red' if data['type'] == 'Prize' else 'gold'
        plt.scatter(data['coords'][0], data['coords'][1], s=200, color=color, edgecolors='black', zorder=5)
        plt.text(data['coords'][0] + 0.1, data['coords'][1] + 0.1, f"{i}: {data['name']}\n(p={data['prize']})")

    path_coords = np.array([nodes_data[node_idx]['coords'] for node_idx in optimal_path])
    plt.plot(path_coords[:, 0], path_coords[:, 1], 'b-o', alpha=0.6, label="最適経路")

    plt.title(f"演習問題2の解\n総スコア: {total_prize:.0f}, 総時間: {total_time:.2f}")
    plt.xlabel("X座標")
    plt.ylabel("Y座標")
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()

elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

-----

## 演習問題3

例題の設定で、スコア地点1 (Spot A) を必ず訪問しなければならないという制約を追加した場合の獲得総スコアの最大値を求めなさい。

###数理モデルの定式化

この問題では、例題のモデル（$T_{max}=9$）に新しい制約を追加する。地点1（Spot A）の訪問を必須とする制約である。
これは、地点1を訪問するか否かを示すバイナリ変数 $y_1$ を強制的に1にすることで表現できる。

  * **追加する制約**:
$$
y_1 = 1
$$

###Python (MIP) による実装

例題のコードに、`y[1] == 1` という制約を追加する。

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, maximize, BINARY, CONTINUOUS, OptimizationStatus
import math
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np

# --- データ定義 ---
# 例題のデータ設定
nodes_data = {
    0: {'name': 'Start', 'coords': (0, 0), 'prize': 0, 'type': 'StartEnd'},
    1: {'name': 'Spot A', 'coords': (1, 3), 'prize': 10, 'type': 'Prize'},
    2: {'name': 'Spot B', 'coords': (4, 4), 'prize': 20, 'type': 'Prize'},
    3: {'name': 'End', 'coords': (5, 1), 'prize': 0, 'type': 'StartEnd'}
}

node_indices = list(nodes_data.keys())
start_node = 0
end_node = 3
prize_nodes = [i for i, data in nodes_data.items() if data['type'] == 'Prize']
prizes = {i: nodes_data[i]['prize'] for i in prize_nodes}

# 移動時間行列の計算
travel_times = {}
for i in node_indices:
    for j in node_indices:
        if i == j: continue
        coord1 = nodes_data[i]['coords']
        coord2 = nodes_data[j]['coords']
        travel_times[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

T_max = 9.0
M = T_max + max(travel_times.values()) + 1

# --- モデル作成 ---
model = Model("Orienteering Problem - Ex3")

# 決定変数
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}") for i in node_indices for j in node_indices if i != j}
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in prize_nodes}
u = {i: model.add_var(lb=0, ub=T_max, name=f"u_{i}") for i in node_indices}

# 目的関数
model.objective = maximize(xsum(prizes[i] * y[i] for i in prize_nodes))

# --- 制約条件 ---
# 基本制約 (演習問題1と同様)
model += xsum(x[start_node, j] for j in node_indices if j != start_node) == 1
model += xsum(x[i, end_node] for i in node_indices if i != end_node) == 1
model += xsum(x[j, start_node] for j in node_indices if j != start_node) == 0
model += xsum(x[end_node, i] for i in node_indices if i != end_node) == 0
for k in prize_nodes:
    model += xsum(x[i, k] for i in node_indices if i != k) == y[k]
    model += xsum(x[k, j] for j in node_indices if j != k) == y[k]
model += u[start_node] == 0
for i in node_indices:
    for j in node_indices:
        if i != j and j != start_node:
            model += u[j] >= u[i] + travel_times[i, j] - M * (1 - x[i, j])

# ★★★ 演習問題3の追加制約 ★★★
model += y[1] == 1, "MustVisit_SpotA"

# --- 最適化の実行 ---
status = model.optimize(max_seconds=60)

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    total_prize = model.objective_value
    print(f"獲得総スコア: {total_prize:.2f}")

    optimal_path = [start_node]
    current_node = start_node
    while current_node != end_node:
        found_next = False
        for j in node_indices:
            if j != current_node and (current_node, j) in x and x[current_node, j].x >= 0.99:
                optimal_path.append(j)
                current_node = j
                found_next = True
                break
        if not found_next: break

    total_time = u[end_node].x
    print(f"最適経路: {optimal_path}")
    print(f"総移動時間: {total_time:.3f} (上限: {T_max})")

    # --- 結果の図示 ---
    plt.figure(figsize=(10, 7))
    for i, data in nodes_data.items():
        color = 'lightgray'
        if i in optimal_path:
            color = 'red' if data['type'] == 'Prize' else 'gold'
        plt.scatter(data['coords'][0], data['coords'][1], s=200, color=color, edgecolors='black', zorder=5)
        plt.text(data['coords'][0] + 0.1, data['coords'][1] + 0.1, f"{i}: {data['name']}\n(p={data['prize']})")

    path_coords = np.array([nodes_data[node_idx]['coords'] for node_idx in optimal_path])
    plt.plot(path_coords[:, 0], path_coords[:, 1], 'b-o', alpha=0.6, label="最適経路")

    plt.title(f"演習問題3の解\n総スコア: {total_prize:.0f}, 総時間: {total_time:.2f}")
    plt.xlabel("X座標")
    plt.ylabel("Y座標")
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()

elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")

-----

## 演習問題4

基本例のスタート地点0とエンド地点3は共通としますが、2つの独立した経路 (2人の旅行者、または2台の車両) でスコア地点を分担して訪問できるとします。各経路の総移動時間上限はそれぞれ $T_{max}=9$ です。スコア地点は一度訪問されればスコアが得られ、複数の経路で同じスコア地点を訪問してもスコアは重複して加算されません。全体の獲得総スコアの最大値を求めなさい。

### 2\. 数理モデルの定式化

これはチームオリエンテーリング問題(Team Orienteering Problem - TOP)の簡単な形式である。経路（車両）が複数になるため、どの経路がどの地点を訪問するかを決定する必要がある。

  * **集合**:
   * $P$: 経路（車両）の集合。$P = \{0, 1\}$
  * **決定変数**:
   * $x_{ij}^p \in \{0, 1\}$: 経路 $p \in P$ がアーク $(i, j)$ を使用する場合に1。
   * $y_k \in \{0, 1\}$: スコア地点 $k \in V_P$ がいずれかの経路で訪問される場合に1。
   * $u_{ip} \ge 0$: 経路 $p$ がノード $i$ に到着する際の累積時間。
  * **目的関数**: 全体で獲得する総スコアを最大化する。
$$
\text{Maximize} \quad \sum_{k \in V_P} p_k y_k
$$
  * **制約条件**:
   1.  **経路ごとの出発・到着**: 各経路 $p \in P$ は、スタート地点から出発し、エンド地点に到着する。
$$
\sum_{j \in V, j \ne s} x_{sj}^p = 1 \quad \forall p \in P
$$
$$
\sum_{i \in V, i \ne e} x_{ie}^p = 1 \quad \forall p \in P
$$
   2.  **経路ごとのフロー保存**: 各経路 $p \in P$ について、
  訪問する中間ノード $k$ でフローが保存される。
$$
\sum_{i \in V, i \ne k} x_{ik}^p - \sum_{j \in V, j \ne k} x_{kj}^p = 0 \quad \forall k \in V \setminus \{s, e\}, \forall p \in P
$$

    3.  **スコア獲得と訪問の関連付け**: スコア地点 $k$ が訪問される($y\_k=1$)のは、いずれかの経路がその地点を訪問する場合である。各スコア地点は高々1つの経路によってのみ訪問される。
        $$
        \sum_{p \in P} \sum_{i \in V, i \ne k} x_{ik}^p = y_k \quad \forall k \in V_P
        $$
        この式は「地点kに入ってくる全経路の合計数が $y\_k$ に等しい」ことを意味する。$y\_k$がバイナリ変数なので、これは「いずれかの経路が1回だけ入ってくる場合に $y\_k=1$」となることを保証する。
   4.  **経路ごとの時間累積(MTZ)**: 各経路 $p$ について、時間累積を計算し部分巡回路を除去する。
   $$
    u_{sp} = 0, \quad \forall p \in P,
   $$
   $$
    u_{jp} \ge u_{ip} + t_{ij} - M(1 - x_{ijp}), \quad \forall (i, j), \forall p \in P, j \ne s
   $$
   5.  **経路ごとの時間上限**: 各経路 $p$ の総移動時間は $T_{max}$ を超えない。
   $$
    u_{ep} \le T_{max}, \quad \forall p \in P
   $$

###Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, maximize, BINARY, CONTINUOUS, OptimizationStatus
import math
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np

# --- データ定義 ---
nodes_data = {
    0: {'name': 'Start', 'coords': (0, 0), 'prize': 0, 'type': 'StartEnd'},
    1: {'name': 'Spot A', 'coords': (1, 3), 'prize': 10, 'type': 'Prize'},
    2: {'name': 'Spot B', 'coords': (4, 4), 'prize': 20, 'type': 'Prize'},
    3: {'name': 'End', 'coords': (5, 1), 'prize': 0, 'type': 'StartEnd'}
}
node_indices = list(nodes_data.keys())
start_node = 0
end_node = 3
prize_nodes = [i for i, data in nodes_data.items() if data['type'] == 'Prize']
prizes = {i: nodes_data[i]['prize'] for i in prize_nodes}
num_vehicles = 2 # 経路(旅行者)の数
vehicles = range(num_vehicles)

travel_times = {}
for i in node_indices:
    for j in node_indices:
        if i == j: continue
        coord1 = nodes_data[i]['coords']
        coord2 = nodes_data[j]['coords']
        travel_times[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

T_max = 9.0
M = T_max + max(travel_times.values()) + 1

# --- モデル作成 ---
model = Model("Team Orienteering Problem - Ex4")

# 決定変数
x = {(i, j, p): model.add_var(var_type=BINARY, name=f"x_{i}_{j}_{p}")
     for i in node_indices for j in node_indices if i != j for p in vehicles}
y = {k: model.add_var(var_type=BINARY, name=f"y_{k}") for k in prize_nodes}
u = {(i, p): model.add_var(lb=0, ub=T_max, name=f"u_{i}_{p}") for i in node_indices for p in vehicles}

# 目的関数
model.objective = maximize(xsum(prizes[k] * y[k] for k in prize_nodes))

# --- 制約条件 ---
for p in vehicles:
    # 1. 各経路はSから出発し、Eへ到着
    model += xsum(x[start_node, j, p] for j in node_indices if j != start_node) == 1
    model += xsum(x[i, end_node, p] for i in node_indices if i != end_node) == 1
    # スタートへ戻らない、エンドから出ない
    model += xsum(x[j, start_node, p] for j in node_indices if j != start_node) == 0
    model += xsum(x[end_node, i, p] for i in node_indices if i != end_node) == 0

    # 2. 各経路のフロー保存則
    for k in prize_nodes:
        model += xsum(x[i, k, p] for i in node_indices if i != k) == xsum(x[k, j, p] for j in node_indices if j != k)

    # 4. 各経路の時間累積(MTZ)と時間上限
    model += u[start_node, p] == 0
    for i in node_indices:
        for j in node_indices:
            if i != j and j != start_node:
                model += u[j, p] >= u[i, p] + travel_times[i, j] - M * (1 - x[i, j, p])

# 3. スコア地点訪問制約 (いずれかの経路が訪問)
for k in prize_nodes:
    # 地点kを訪問する経路は最大1つ (y_kで表現)
    model += xsum(x[i, k, p] for i in node_indices if i != k for p in vehicles) == y[k]

# --- 最適化の実行 ---
status = model.optimize(max_seconds=60)

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    total_prize = model.objective_value
    print(f"獲得総スコア: {total_prize:.2f}")

    # --- 結果の図示 ---
    plt.figure(figsize=(10, 8))
    colors = ['blue', 'green']
    # 地点プロット
    visited_nodes = set()
    for p in vehicles:
        for i, j,p in x:
            if x[i, j, p].x > 0.99:
                visited_nodes.add(i)
                visited_nodes.add(j)

    for i, data in nodes_data.items():
        color = 'lightgray'
        if i in visited_nodes:
            color = 'red' if data['type'] == 'Prize' else 'gold'
        plt.scatter(data['coords'][0], data['coords'][1], s=200, color=color, edgecolors='black', zorder=5)
        plt.text(data['coords'][0] + 0.1, data['coords'][1] + 0.1, f"{i}: {data['name']}\n(p={data['prize']})")

    # 経路ごとの結果表示とプロット
    for p in vehicles:
        path = [start_node]
        current_node = start_node
        path_time = 0

        # 経路が何かを訪問しているかチェック
        if sum(x[start_node, j, p].x for j in node_indices if j != start_node) < 0.5:
            print(f"\n経路 {p+1} は使用されませんでした (Start->Endの直行)。")
            continue

        while current_node != end_node:
            for j in node_indices:
                if j != current_node and x[current_node, j, p].x > 0.99:
                    path.append(j)
                    path_time += travel_times[current_node, j]
                    current_node = j
                    break

        print(f"\n経路 {p+1}: {path}")
        print(f"  総移動時間: {u[end_node, p].x:.3f} (上限: {T_max})")

        path_coords = np.array([nodes_data[node_idx]['coords'] for node_idx in path])
        plt.plot(path_coords[:, 0], path_coords[:, 1], '-o', color=colors[p], alpha=0.7, label=f"経路 {p+1}")

    plt.title(f"演習問題4の解\n総スコア: {total_prize:.0f}")
    plt.xlabel("X座標")
    plt.ylabel("Y座標")
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()

else:
    print("解が見つかりませんでした。")

-----

## 演習問題5

###問題の確認

基本例において、スコア地点1 (Spot A) は累積時間 $[2, 5]$ の間に、スコア地点2 (Spot B) は累積時間 $[4, 9]$ の間に訪問しなければならないという時間枠制約を追加します。訪問しない場合はこの制約は関係ありません。この条件下で最適な経路、獲得総スコア、総移動時間を求めなさい。

### 2\. 数理モデルの定式化

この問題は、オリエンテーリング問題に時間枠(Time Windows)制約を追加するものである。これはOPTW (Orienteering Problem with Time Windows) と呼ばれる問題の変種である。
特定のスコア地点 $k$ を訪問する場合($y_k=1$)、その地点への到着時刻 $u_k$ が指定された時間枠 $[L_k, U_k]$ 内になければならない。

  * **追加する制約**:
      * 地点1(Spot A)の時間枠: $[L_1, U_1] = [2, 5]$
      * 地点2(Spot B)の時間枠: $[L_2, U_2] = [4, 9]$

これを数式で表現すると、訪問する場合 ($y_k=1$) のみ制約が有効になるようにモデル化する必要がある。

$$
L_k \cdot y_k \le u_k \quad \forall k \in V_{TW}
$$
$$
u_k \le U_k \cdot y_k + M \cdot (1 - y_k) \quad \forall k \in V_{TW}
$$

ここで $V_{TW}$ は時間枠制約を持つノードの集合、
$M$は大きな数である。

もし $y_k=1$ なら、$L_k \le u_k \le U_k$ となる。
もし $y_k=0$ なら、$0 \le u_k \le M$ となり、
時間枠制約は事実上無効化される。

###Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, maximize, BINARY, CONTINUOUS, OptimizationStatus
import math
import matplotlib.pyplot as plt
import japanize_matplotlib
import numpy as np

# --- データ定義 ---
nodes_data = {
    0: {'name': 'Start', 'coords': (0, 0), 'prize': 0, 'type': 'StartEnd'},
    1: {'name': 'Spot A', 'coords': (1, 3), 'prize': 10, 'type': 'Prize'},
    2: {'name': 'Spot B', 'coords': (4, 4), 'prize': 20, 'type': 'Prize'},
    3: {'name': 'End', 'coords': (5, 1), 'prize': 0, 'type': 'StartEnd'}
}
node_indices = list(nodes_data.keys())
num_nodes = len(node_indices)
start_node = 0
end_node = 3
prize_nodes = [i for i, data in nodes_data.items() if data['type'] == 'Prize']
prizes = {i: nodes_data[i]['prize'] for i in prize_nodes}

# 時間枠制約
time_windows = {
    1: {'lower': 2.0, 'upper': 5.0},
    2: {'lower': 4.0, 'upper': 9.0}
}

travel_times = {}
for i in node_indices:
    for j in node_indices:
        if i == j: continue
        coord1 = nodes_data[i]['coords']
        coord2 = nodes_data[j]['coords']
        travel_times[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

T_max = 9.0
# ビッグMは十分に大きな値であれば良い。ここではT_maxで代用可能。
M = T_max

# --- モデル作成 ---
model = Model("Orienteering Problem with Time Windows - Ex5 Corrected")

# 決定変数
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}") for i in node_indices for j in node_indices if i != j}
y = {i: model.add_var(var_type=BINARY, name=f"y_{i}") for i in prize_nodes}
u = {i: model.add_var(lb=0, ub=T_max, name=f"u_{i}") for i in node_indices}

# 目的関数
model.objective = maximize(xsum(prizes[i] * y[i] for i in prize_nodes))

# --- 制約条件 ---
# 基本制約
model += xsum(x[start_node, j] for j in node_indices if j != start_node) == 1, "LeaveStart"
model += xsum(x[i, end_node] for i in node_indices if i != end_node) == 1, "EnterEnd"
model += xsum(x[j, start_node] for j in node_indices if j != start_node) == 0, "NoEnterStart"
model += xsum(x[end_node, i] for i in node_indices if i != end_node) == 0, "NoLeaveEnd"

for k in prize_nodes:
    model += xsum(x[i, k] for i in node_indices if i != k) == y[k], f"FlowIn_{k}"
    model += xsum(x[k, j] for j in node_indices if j != k) == y[k], f"FlowOut_{k}"

model += u[start_node] == 0, "StartTime"
# MTZ制約のMはT_maxより大きい必要があるため別途定義
M_mtz = T_max + max(travel_times.values())
for i in node_indices:
    for j in node_indices:
        if i != j and j != start_node:
            model += u[j] >= u[i] + travel_times[i, j] - M_mtz * (1 - x[i, j]), f"TimeSubtour_{i}_{j}"

# ★★★ 演習問題5の修正された追加制約 (時間枠) ★★★
for k, tw in time_windows.items():
    # 訪問する場合(y[k]=1)、到着時刻u[k]は時間枠の下限以上でなければならない。
    # 訪問しない場合(y[k]=0)、u[k]>=0となり、この制約は常に満たされる。
    model += u[k] >= tw['lower'] * y[k], f"TW_Lower_{k}"

    # 訪問する場合(y[k]=1)、到着時刻u[k]は時間枠の上限以下でなければならない。
    # 訪問しない場合(y[k]=0)、u[k] <= M となり、uの定義(ub=T_max)からこの制約は常に満たされる。
    model += u[k] <= tw['upper'] * y[k] + M * (1 - y[k]), f"TW_Upper_{k}"


# --- 最適化の実行 ---
status = model.optimize(max_seconds=60)

# --- 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    total_prize = model.objective_value
    print(f"獲得総スコア: {total_prize:.2f}")

    optimal_path = [start_node]
    current_node = start_node
    while current_node != end_node and len(optimal_path) <= num_nodes:
        found_next = False
        for j in node_indices:
            if j != current_node and (current_node, j) in x and x[current_node, j].x >= 0.99:
                optimal_path.append(j)
                current_node = j
                found_next = True
                break
        if not found_next: break

    total_time = u[end_node].x
    print(f"最適経路: {optimal_path}")
    print(f"総移動時間: {total_time:.3f} (上限: {T_max})")

    print("\n各地点の訪問時刻:")
    for i in node_indices:
         # 訪問したノード、または少しでも値を持つノードの時刻を表示
        if u[i].x is not None and u[i].x > 1e-6 or i in optimal_path:
             print(f"  ノード {i} ({nodes_data[i]['name']}): 到着時刻 {u[i].x:.3f}")

    # --- 結果の図示 ---
    plt.figure(figsize=(10, 7))
    for i, data in nodes_data.items():
        color = 'lightgray'
        if i in optimal_path:
            color = 'red' if data['type'] == 'Prize' else 'gold'
        plt.scatter(data['coords'][0], data['coords'][1], s=200, color=color, edgecolors='black', zorder=5)
        text = f"{i}: {data['name']}\n(p={data['prize']})"
        if i in time_windows:
            text += f"\nTW: [{time_windows[i]['lower']:.0f}, {time_windows[i]['upper']:.0f}]"
        plt.text(data['coords'][0] + 0.1, data['coords'][1] - 0.2, text)

    path_coords = np.array([nodes_data[node_idx]['coords'] for node_idx in optimal_path])
    plt.plot(path_coords[:, 0], path_coords[:, 1], 'b-o', alpha=0.6, label="最適経路")

    plt.title(f"演習問題5の解\n総スコア: {total_prize:.0f}, 総時間: {total_time:.2f}")
    plt.xlabel("X座標")
    plt.ylabel("Y座標")
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()

elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能な問題です。制約を満たす解が存在しません。")
else:
    print(f"最適化が停止しました。ステータス: {status}")